In [75]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import sys
sys.path.insert(0, "./../../")
from src.data_loader import add_recent_stats, load_game_data_zan, add_season_rankings
pd.set_option('display.max_columns', None)

%reload_ext autoreload
%autoreload 2

In [129]:
# game_data = load_game_data_zan()
# game_data_recent = add_recent_stats(game_data, recent_range=15)

In [35]:
path_game_data_recent = "/home/matej/Documents/Projects/nba-data-mining/data/processed/game-data-recent-15.csv"
game_data_recent = pd.read_csv(path_game_data_recent)
game_data_recent = add_season_rankings(game_data_recent)
game_data_recent = game_data_recent[game_data_recent["season_name"] > 2000]
game_data_recent = game_data_recent[(game_data_recent["home_rank"] > 0) & (game_data_recent["visitor_rank"] > 0)]

* * *
### Analysis of features (AFTER GAME)

Following pairs of features are highly correlated:
- home_final_score & home_fg_made
- visitor_final_score & visitor_fg_made
- home_final_score & home_TSP
- visitor_final_score & visitor_TSP
- home_rebound & visitor_fg_missed
- visitor_rebound & home_fg_missed

Suggested features to be included in the model:
- home_TSP
- home_fg_made (????)
- home_timeout
- visitor TSP
- visitor_fg_made (????)
- visitor_timeout
- home_final_score_diff


In [36]:
correlations = game_data_recent.corr()
correlations_home_win = correlations["home_win"].sort_values()
correlations_home_win[correlations_home_win > 0.2]

home_rebound             0.241544
visitor_timeout          0.285803
visitor_fg_missed        0.327362
home_fg_made             0.346392
home_final_score         0.416843
home_TSP                 0.452848
home_final_score_diff    0.802578
home_win                 1.000000
Name: home_win, dtype: float64

In [38]:
correlations_positive = game_data_recent[["home_rebound", "visitor_timeout", "visitor_fg_missed", "home_fg_made", "home_final_score", "home_TSP", "home_final_score_diff"]].corr()

fig = px.imshow(correlations_positive)
fig.show()

In [39]:
correlations_home_win[correlations_home_win < -0.2]

visitor_final_score_diff        -0.802578
visitor_TSP                     -0.453650
home_timeout                    -0.421919
visitor_final_score             -0.413824
visitor_fg_made                 -0.338465
home_fg_missed                  -0.324381
visitor_scoring_leader_points   -0.235978
visitor_rebound                 -0.232083
Name: home_win, dtype: float64

In [40]:
correlations_negative = game_data_recent[["visitor_rebound", "home_timeout", "home_fg_missed", "visitor_fg_made", "visitor_final_score", "visitor_TSP", "visitor_final_score_diff", "visitor_scoring_leader_points"]].corr()

fig = px.imshow(correlations_negative)
fig.show()

* * *
### Analysis of recent features (BEFORE GAME)

Following pairs of recent features are highly correlated:
- home_recent_fg_made & home_recent_points
- home_recent_fg_made & home_recent_3PT_made
- home_recent_3PT_made & home_recent_points
- home_recent_TSP & home_recent_ft_made

Suggested recent features to be included in the model:
- home_recent_TSP
- home_recent_timeout
- home_recent_points
- home_rank
- visitor_recent_TSP
- visitor_recent_timeout
- visitor_recent_points
- visitor_rank

In [44]:
data_recent = game_data_recent.loc[:, "home_recent_home_game_ratio" : "visitor_rank"].copy()
data_recent["home_win"] = game_data_recent["home_win"]

In [45]:
correlations_recent = data_recent.corr()
correlations_recent_home_win = correlations_recent["home_win"].sort_values()
correlations_recent_home_win[correlations_recent_home_win > 0.04]

home_recent_ft_made         0.049301
home_recent_3PT_made        0.049588
visitor_recent_foul         0.050640
home_recent_rebound         0.057152
visitor_recent_turnover     0.067339
home_recent_fg_made         0.078980
visitor_recent_fg_missed    0.085874
home_recent_points          0.096128
visitor_recent_timeout      0.102315
home_recent_TSP             0.129539
visitor_rank                0.183250
home_win                    1.000000
Name: home_win, dtype: float64

In [46]:
correlations_recent_subset = data_recent[["visitor_recent_foul", "home_recent_ft_made", "home_recent_3PT_made", "home_recent_rebound",
                                            "visitor_recent_turnover", "home_recent_fg_made", "visitor_recent_fg_missed", "home_recent_points",
                                            "visitor_recent_timeout", "home_recent_TSP", "home_rank", "visitor_rank"]].corr()

fig = px.imshow(correlations_recent_subset, width=700, height=700)
fig.show()                                            

## Predictions

In [115]:
scaler = MinMaxScaler()
label_encoder = LabelEncoder()
model = LogisticRegression(solver='saga', max_iter=1000, C=0.11)

### Prediction before the game

In [58]:
# last three seasons
cols = ["home_team_id", "visitor_team_id", "home_recent_TSP", "visitor_recent_TSP", "home_recent_timeout", "visitor_recent_timeout",
        "home_recent_points", "visitor_recent_points", "home_rank", "visitor_rank", "home_win"]

to_normalize = ["home_recent_TSP", "home_recent_timeout", "visitor_recent_TSP", "visitor_recent_timeout", "home_recent_points", "visitor_recent_points", "home_rank", "visitor_rank"]

data = game_data_recent[game_data_recent["season_name"] >= 2014][cols].copy()

data["home_team_id"] = label_encoder.fit_transform(game_data_recent[game_data_recent["season_name"] >= 2014]["home_team_id"])
data["visitor_team_id"] = label_encoder.fit_transform(game_data_recent[game_data_recent["season_name"] >= 2014]["visitor_team_id"])
data[to_normalize] = scaler.fit_transform(data[to_normalize])
data

,home_team_id,visitor_team_id,home_recent_TSP,visitor_recent_TSP,home_recent_timeout,visitor_recent_timeout,home_recent_points,visitor_recent_points,home_rank,visitor_rank,home_win
16815,3,16,0.696852,0.506770,0.50,0.50,0.317073,0.195122,0.652174,0.913043,1
16816,22,5,0.532789,0.597526,0.25,1.00,0.512195,0.512195,0.000000,0.304348,1
16817,10,8,0.530567,0.719699,0.50,0.50,0.512195,0.658537,0.826087,0.173913,0
16818,29,12,0.464036,0.767927,0.50,0.75,0.365854,0.317073,0.434783,1.000000,1
16819,17,18,0.376304,0.442706,0.75,0.75,0.024390,0.341463,0.130435,0.956522,1
...,...,...,...,...,...,...,...,...,...,...,...
22960,12,23,0.593213,0.253085,0.50,0.50,0.853659,0.560976,0.434783,0.304348,0
22961,22,5,0.581376,0.420659,0.50,0.75,0.609756,0.682927,0.347826,0.869565,1
22962,6,13,0.294328,0.467019,0.50,0.50,0.439024,0.585366,0.391304,0.347826,1
22963,9,25,0.824276,0.774478,0.50,0.25,0.756098,0.780488,0.521739,0.304348,1


In [133]:
# all seasons - all features
data = game_data_recent.loc[:,"home_recent_points":"visitor_rank"].copy()
data["home_team_id"] = label_encoder.fit_transform(game_data_recent["home_team_id"])
data["visitor_team_id"] = label_encoder.fit_transform(game_data_recent["visitor_team_id"])
data["home_win"] = game_data_recent["home_win"]
data.loc[:,"home_recent_points":"visitor_rank"] = scaler.fit_transform(data.loc[:,"home_recent_points":"visitor_rank"])
data

,home_recent_points,home_recent_timeout,home_recent_TSP,home_recent_fg_made,home_recent_fg_missed,home_recent_3PT_made,home_recent_3PT_missed,home_recent_ft_made,home_recent_ft_missed,home_recent_players_deployed,home_recent_rebound,home_recent_turnover,home_recent_foul,visitor_recent_home_game_ratio,visitor_recent_win_ratio,visitor_recent_points,visitor_recent_timeout,visitor_recent_TSP,visitor_recent_fg_made,visitor_recent_fg_missed,visitor_recent_3PT_made,visitor_recent_3PT_missed,visitor_recent_ft_made,visitor_recent_ft_missed,visitor_recent_players_deployed,visitor_recent_rebound,visitor_recent_turnover,visitor_recent_foul,home_rank,visitor_rank,home_team_id,visitor_team_id,home_win
1189,0.411765,0.6,0.614958,0.368421,0.428571,0.117647,0.035714,0.722222,0.333333,0.6,0.444444,0.583333,0.533333,0.0,0.0,0.431373,0.4,0.413002,0.30,0.476190,0.411765,0.392857,0.555556,0.5,0.6,0.315789,0.461538,0.600000,0.692308,0.576923,2,1,0
1190,0.254902,0.6,0.421491,0.368421,0.333333,0.176471,0.142857,0.333333,0.222222,0.4,0.388889,0.500000,0.600000,0.0,0.0,0.411765,0.6,0.729988,0.40,0.238095,0.058824,0.000000,0.722222,0.6,0.4,0.315789,0.692308,0.400000,0.269231,0.846154,15,27,1
1191,0.294118,0.4,0.367045,0.315789,0.571429,0.117647,0.178571,0.500000,0.222222,0.4,0.333333,0.583333,0.533333,0.0,0.0,0.431373,0.4,0.521625,0.45,0.428571,0.176471,0.142857,0.555556,0.4,0.6,0.526316,0.538462,0.600000,0.730769,0.500000,14,17,1
1192,0.372549,0.4,0.416921,0.421053,0.428571,0.294118,0.214286,0.333333,0.444444,0.6,0.277778,0.500000,0.666667,0.0,0.0,0.450980,0.4,0.363993,0.50,0.571429,0.235294,0.178571,0.333333,0.4,0.4,0.473684,0.384615,0.400000,0.461538,0.307692,16,24,1
1193,0.450980,0.4,0.524950,0.578947,0.476190,0.117647,0.035714,0.388889,0.333333,0.6,0.444444,0.333333,0.666667,0.0,0.0,0.274510,0.4,0.317161,0.30,0.619048,0.058824,0.071429,0.500000,0.5,0.6,0.684211,0.461538,0.400000,0.307692,0.038462,13,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22960,0.882353,0.4,0.502180,0.789474,0.619048,0.705882,0.714286,0.555556,0.333333,0.6,0.944444,0.333333,0.333333,0.0,0.0,0.647059,0.4,0.210333,0.60,0.809524,0.647059,0.678571,0.333333,0.5,0.6,0.736842,0.461538,0.466667,0.384615,0.269231,12,23,0
22961,0.686275,0.4,0.492815,0.789474,0.476190,0.470588,0.428571,0.333333,0.111111,0.6,0.555556,0.166667,0.200000,0.0,0.0,0.745098,0.6,0.345632,0.65,0.666667,0.705882,0.785714,0.500000,0.4,0.8,0.684211,0.230769,0.466667,0.307692,0.769231,22,5,1
22962,0.549020,0.4,0.265723,0.631579,0.619048,0.529412,0.607143,0.222222,0.222222,0.6,0.555556,0.250000,0.400000,0.0,0.0,0.666667,0.4,0.383063,0.65,0.619048,0.529412,0.535714,0.444444,0.4,0.6,0.473684,0.384615,0.333333,0.346154,0.307692,6,13,1
22963,0.803922,0.4,0.684981,0.736842,0.476190,0.588235,0.428571,0.722222,0.444444,0.8,0.611111,0.416667,0.600000,0.0,0.0,0.823529,0.2,0.631306,0.75,0.333333,0.705882,0.571429,0.500000,0.5,0.8,0.736842,0.538462,0.466667,0.461538,0.269231,9,25,1


In [116]:
# all seasons - few features
cols = ["home_team_id", "visitor_team_id", "home_recent_TSP", "visitor_recent_TSP", "home_recent_timeout", "visitor_recent_timeout",
        "home_recent_points", "visitor_recent_points", "home_rank", "visitor_rank", "home_win"]

to_normalize = ["home_recent_TSP", "home_recent_timeout", "visitor_recent_TSP", "visitor_recent_timeout", "home_recent_points", "visitor_recent_points", "home_rank", "visitor_rank"]

data = game_data_recent[cols].copy()
data["home_team_id"] = label_encoder.fit_transform(game_data_recent["home_team_id"])
data["visitor_team_id"] = label_encoder.fit_transform(game_data_recent["visitor_team_id"])
data[to_normalize] = scaler.fit_transform(data[to_normalize])
data

,home_team_id,visitor_team_id,home_recent_TSP,visitor_recent_TSP,home_recent_timeout,visitor_recent_timeout,home_recent_points,visitor_recent_points,home_rank,visitor_rank,home_win
1189,2,1,0.614958,0.413002,0.6,0.4,0.411765,0.431373,0.692308,0.576923,0
1190,15,27,0.421491,0.729988,0.6,0.6,0.254902,0.411765,0.269231,0.846154,1
1191,14,17,0.367045,0.521625,0.4,0.4,0.294118,0.431373,0.730769,0.500000,1
1192,16,24,0.416921,0.363993,0.4,0.4,0.372549,0.450980,0.461538,0.307692,1
1193,13,18,0.524950,0.317161,0.4,0.4,0.450980,0.274510,0.307692,0.038462,1
...,...,...,...,...,...,...,...,...,...,...,...
22960,12,23,0.502180,0.210333,0.4,0.4,0.882353,0.647059,0.384615,0.269231,0
22961,22,5,0.492815,0.345632,0.4,0.6,0.686275,0.745098,0.307692,0.769231,1
22962,6,13,0.265723,0.383063,0.4,0.4,0.549020,0.666667,0.346154,0.307692,1
22963,9,25,0.684981,0.631306,0.4,0.2,0.803922,0.823529,0.461538,0.269231,1


In [134]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["home_win"]), data["home_win"], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

print("Train dataset: {}%".format((X_train.shape[0] / data.shape[0]) * 100))
print("Validate dataset: {}%".format((X_val.shape[0] / data.shape[0]) * 100))
print("Test dataset: {}%".format((X_test.shape[0] / data.shape[0]) * 100))

Train dataset: 59.99440611597986%
Validate dataset: 20.002796942010068%
Test dataset: 20.002796942010068%


In [114]:
grid = {"C":np.arange(0.01, 5, 0.1), "penalty":["l2"]}
logreg_cv = GridSearchCV(model, grid, cv=10)
logreg_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 0.11, 'penalty': 'l2'}
accuracy : 0.6413364413364413


In [135]:
model.fit(X_train, y_train)
predictions = model.predict(X_val)

fpr, tpr, thresholds = roc_curve(y_val, predictions)
print(classification_report(y_val, predictions))

              precision    recall  f1-score   support

           0       0.59      0.40      0.47      1703
           1       0.67      0.82      0.74      2588

    accuracy                           0.65      4291
   macro avg       0.63      0.61      0.61      4291
weighted avg       0.64      0.65      0.63      4291



In [136]:
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

### Prediction after the game

In [86]:
data = game_data_recent[["home_team_id", "visitor_team_id", "home_TSP", "home_timeout", "visitor_TSP", "visitor_timeout", "home_win"]].copy()
data["home_team_id"] = label_encoder.fit_transform(game_data_recent["home_team_id"])
data["visitor_team_id"] = label_encoder.fit_transform(game_data_recent["visitor_team_id"])
data[["home_TSP", "home_timeout", "visitor_TSP", "visitor_timeout"]] = scaler.fit_transform(data[["home_TSP", "home_timeout", "visitor_TSP", "visitor_timeout"]])
data

,home_team_id,visitor_team_id,home_TSP,home_timeout,visitor_TSP,visitor_timeout,home_win
0,15,18,0.294900,0.400000,0.750004,0.117647,0
1,14,2,0.255317,0.466667,0.399886,0.294118,0
2,16,27,0.440890,0.266667,0.475567,0.352941,1
3,0,29,0.271960,0.266667,0.887694,0.235294,0
4,24,28,0.280250,0.400000,0.397672,0.235294,0
...,...,...,...,...,...,...,...
22960,12,23,0.297271,0.333333,0.376546,0.117647,0
22961,22,5,0.386106,0.333333,0.235184,0.294118,1
22962,6,13,0.306490,0.266667,0.260060,0.352941,1
22963,9,25,0.512595,0.333333,0.460439,0.352941,1


In [88]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["home_win"]), data["home_win"], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

print("Train dataset: {}%".format((X_train.shape[0] / data.shape[0]) * 100))
print("Validate dataset: {}%".format((X_val.shape[0] / data.shape[0]) * 100))
print("Test dataset: {}%".format((X_test.shape[0] / data.shape[0]) * 100))

Train dataset: 60.0%
Validate dataset: 20.0%
Test dataset: 20.0%


In [89]:
model.fit(X_train, y_train)
predictions = model.predict(X_val)

fpr, tpr, thresholds = roc_curve(y_val, predictions)
print(classification_report(y_val, predictions))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      1842
           1       0.89      0.91      0.90      2751

    accuracy                           0.88      4593
   macro avg       0.88      0.87      0.87      4593
weighted avg       0.88      0.88      0.88      4593



In [120]:
game_data_recent

,GAME_ID,play_count,home_team_id,visitor_team_id,home_record_wins,home_record_losses,season_name,visitor_team_city,visitor_team_nickname,home_final_score,visitor_final_score,home_win,home_team_city,home_team_nickname,periods,minutes_played,visitor_players_deployed_ids,visitor_players_deployed,home_players_deployed_ids,home_players_deployed,visitor_fg_made,visitor_fg_missed,visitor_3PT_made,visitor_3PT_missed,home_fg_made,home_fg_missed,home_3PT_made,home_3PT_missed,visitor_ft_made,visitor_ft_missed,home_ft_made,home_ft_missed,visitor_rebound,home_rebound,visitor_team_rebound,home_team_rebound,visitor_turnover,home_turnover,visitor_team_turnover,home_team_turnover,visitor_foul,home_foul,visitor_subs,home_subs,visitor_timeout,home_timeout,visitor_jump_balls_won,home_jump_balls_won,tip_off_winner,visitor_ejection,home_ejection,visitor_team_ejection,home_team_ejection,home_scoring_leader,home_scoring_leader_points,visitor_scoring_leader,visitor_scoring_leader_points,home_made_max_shot_distance,visitor_made_max_shot_distance,home_made_min_shot_distance,visitor_made_min_shot_distance,home_made_mean_shot_distance,visitor_made_mean_shot_distance,visitor_record_wins,visitor_record_losses,games_already_played_in_season,home_TSP,visitor_TSP,home_final_score_diff,visitor_final_score_diff,home_common_lineup,visitor_common_lineup,home_recent_home_game_ratio,home_recent_win_ratio,home_recent_points,home_recent_timeout,home_recent_TSP,home_recent_fg_made,home_recent_fg_missed,home_recent_3PT_made,home_recent_3PT_missed,home_recent_ft_made,home_recent_ft_missed,home_recent_players_deployed,home_recent_rebound,home_recent_turnover,home_recent_foul,visitor_recent_home_game_ratio,visitor_recent_win_ratio,visitor_recent_points,visitor_recent_timeout,visitor_recent_TSP,visitor_recent_fg_made,visitor_recent_fg_missed,visitor_recent_3PT_made,visitor_recent_3PT_missed,visitor_recent_ft_made,visitor_recent_ft_missed,visitor_recent_players_deployed,visitor_recent_rebound,visitor_recent_turnover,visitor_recent_foul,home_rank,visitor_rank
1189,20100001,448,1610612739,1610612738,0,1,2001,Boston,Celtics,89,108,0,Cleveland,Cavaliers,4,48,"[{2208, 962, 677, 72, 1960, 2218, 1548, 1718, ...",12,"[{1889, 1730, 1729, 1892, 902, 179, 2036, 1908...",12,36,45,8,10,39,51,3,14,28,3,8,2,48,36,7,7,14,13,1,1,15,22,26,21,3,6,1,1,HOME_PLAYER,0,0,0,0,445,13,1718,29,27,26,0,0,12.435897,11.166667,1,0,1,0.456504,0.607287,-19.0,19.0,0,0,0,0,97,6,0.579838,35,44,3,5,22,5,10,42,15,22,0,0,98,5,0.543054,34,45,8,15,19,6,10,39,13,23,19,16
1190,20100002,455,1610612752,1610612764,1,0,2001,Washington,Wizards,93,91,1,New York,Knicks,4,48,"[{1888, 363, 43, 461, 1838, 2198, 1751, 2042, ...",9,"[{1000, 970, 369, 275, 84, 280, 221, 349, 703}]",9,36,46,6,7,32,43,6,14,13,5,23,10,39,46,5,7,14,17,1,0,27,21,16,18,8,6,2,0,VISITOR_PLAYER,0,1,0,0,84,28,893,19,26,27,0,0,11.843750,12.500000,0,1,1,0.554893,0.518696,2.0,-2.0,0,0,0,0,89,6,0.540731,35,42,4,8,15,4,9,41,14,23,0,0,97,6,0.606619,36,40,2,4,22,7,9,39,16,20,8,23
1191,20100003,490,1610612751,1610612754,1,0,2001,Indiana,Pacers,103,97,1,New Jersey,Nets,4,48,"[{1506, 1733, 397, 1902, 2224, 979, 147, 696, ...",9,"[{1760, 2210, 1928, 2030, 1425, 467, 1496, 954...",9,37,44,6,9,37,58,6,10,17,3,23,8,42,47,12,6,20,11,2,0,25,23,26,17,7,5,0,1,HOME_PLAYER,0,0,0,0,1496,26,147,43,25,26,0,0,10.459459,13.594595,0,1,1,0.504704,0.553653,6.0,-6.0,1,1,0,0,91,5,0.529725,34,47,3,9,18,4,9,40,15,22,0,0,98,5,0.564836,37,44,4,8,19,5,10,43,14,23,20,14
1192,20100004,484,1610612753,1610612761,1,0,2001,Toronto,Raptors,114,85,1,Orlando,Magic,4,48,"[{2050, 2306, 164, 1541, 165, 966, 2214, 1713,...",12,"[{448, 353, 255, 2212, 1607, 42, 270, 1727, 69...",12,27,48,3,9,44,28,4,4,28,17,22,9,38,37,8,5,21,17,0,2,29,32,16,20,6,4,1,0,VISITOR_PLAYER,0,0,0,0,1503,23,164,13,26,25,0,0,9.204545,10.555556,0,1,1,0.754767,0.529397,29.0,-29.0,0,1,0,0,95,5,0.539807,36,44,6,10,15,6,10,39,14,24,0,0,99,5,0.533226,38,47,5,9,15,5,9,42,12,20,13,9
1193,20100005,429,1610612750,1610612755,1,0,2001,Philadel

In [129]:
data

,home_recent_points,home_recent_timeout,home_recent_TSP,home_recent_fg_made,home_recent_fg_missed,home_recent_3PT_made,home_recent_3PT_missed,home_recent_ft_made,home_recent_ft_missed,home_recent_players_deployed,home_recent_rebound,home_recent_turnover,home_recent_foul,visitor_recent_home_game_ratio,visitor_recent_win_ratio,visitor_recent_points,visitor_recent_timeout,visitor_recent_TSP,visitor_recent_fg_made,visitor_recent_fg_missed,visitor_recent_3PT_made,visitor_recent_3PT_missed,visitor_recent_ft_made,visitor_recent_ft_missed,visitor_recent_players_deployed,visitor_recent_rebound,visitor_recent_turnover,visitor_recent_foul,home_rank,visitor_rank,home_team_id,visitor_team_id
1189,0.411765,0.6,0.614958,0.368421,0.428571,0.117647,0.035714,0.722222,0.333333,0.6,0.444444,0.583333,0.533333,0.0,0.0,0.431373,0.4,0.413002,0.30,0.476190,0.411765,0.392857,0.555556,0.5,0.6,0.315789,0.461538,0.600000,0.692308,0.576923,2,1
1190,0.254902,0.6,0.421491,0.368421,0.333333,0.176471,0.142857,0.333333,0.222222,0.4,0.388889,0.500000,0.600000,0.0,0.0,0.411765,0.6,0.729988,0.40,0.238095,0.058824,0.000000,0.722222,0.6,0.4,0.315789,0.692308,0.400000,0.269231,0.846154,15,27
1191,0.294118,0.4,0.367045,0.315789,0.571429,0.117647,0.178571,0.500000,0.222222,0.4,0.333333,0.583333,0.533333,0.0,0.0,0.431373,0.4,0.521625,0.45,0.428571,0.176471,0.142857,0.555556,0.4,0.6,0.526316,0.538462,0.600000,0.730769,0.500000,14,17
1192,0.372549,0.4,0.416921,0.421053,0.428571,0.294118,0.214286,0.333333,0.444444,0.6,0.277778,0.500000,0.666667,0.0,0.0,0.450980,0.4,0.363993,0.50,0.571429,0.235294,0.178571,0.333333,0.4,0.4,0.473684,0.384615,0.400000,0.461538,0.307692,16,24
1193,0.450980,0.4,0.524950,0.578947,0.476190,0.117647,0.035714,0.388889,0.333333,0.6,0.444444,0.333333,0.666667,0.0,0.0,0.274510,0.4,0.317161,0.30,0.619048,0.058824,0.071429,0.500000,0.5,0.6,0.684211,0.461538,0.400000,0.307692,0.038462,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22960,0.882353,0.4,0.502180,0.789474,0.619048,0.705882,0.714286,0.555556,0.333333,0.6,0.944444,0.333333,0.333333,0.0,0.0,0.647059,0.4,0.210333,0.60,0.809524,0.647059,0.678571,0.333333,0.5,0.6,0.736842,0.461538,0.466667,0.384615,0.269231,12,23
22961,0.686275,0.4,0.492815,0.789474,0.476190,0.470588,0.428571,0.333333,0.111111,0.6,0.555556,0.166667,0.200000,0.0,0.0,0.745098,0.6,0.345632,0.65,0.666667,0.705882,0.785714,0.500000,0.4,0.8,0.684211,0.230769,0.466667,0.307692,0.769231,22,5
22962,0.549020,0.4,0.265723,0.631579,0.619048,0.529412,0.607143,0.222222,0.222222,0.6,0.555556,0.250000,0.400000,0.0,0.0,0.666667,0.4,0.383063,0.65,0.619048,0.529412,0.535714,0.444444,0.4,0.6,0.473684,0.384615,0.333333,0.346154,0.307692,6,13
22963,0.803922,0.4,0.684981,0.736842,0.476190,0.588235,0.428571,0.722222,0.444444,0.8,0.611111,0.416667,0.600000,0.0,0.0,0.823529,0.2,0.631306,0.75,0.333333,0.705882,0.571429,0.500000,0.5,0.8,0.736842,0.538462,0.466667,0.461538,0.269231,9,25
